<a href="https://colab.research.google.com/github/WangKehanK/CS505/blob/main/Twitter_User_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import json
import os

In [3]:
% cd /content/drive/MyDrive

/content/drive/MyDrive


In [4]:
df = pd.read_csv("Twitter_user_handles_to_predict.csv", encoding = "utf-8")
df_labeled = pd.read_csv("Twitter_users_labeled_with_age_and_gender.csv", encoding = "latin-1")

In [ ]:
df_no_na = df[df['Username'].notna()]
df_no_na

,ID,Username
0,1,JayHolz410
1,2,kelechief
2,3,VicSpencer
3,4,ItsAlexDodson
4,5,xopinkvodka6
...,...,...
25124,25125,13luvr
25125,25126,Racetac
25126,25127,MrNoFilter_100
25127,25128,OgheneNerojr


In [ ]:
age_no_na = df_labeled[df_labeled['human.labeled.age'].notna()]
age_no_na

,Unnamed: 0,screen_name,user_id,lang,name,location,description,protected,followers_count,friends_count,statuses_count,favourites_count,account_created_at,verified,profile_banner_url,profile_background_url,profile_image_url,user.name,num.tweets.used.Lexicon.prediction,Lexicon.age.prediction,Lexicon.gender.prediction..index.,lexicon.gender.prediction,human.labeled.gender,human.labeled.age
0,1,_____zac_____,4.614412e+08,en,zac ¢,"Maryland, USA",_____Û___È_Ü´Ù,False,208,178,27912,4179,1/11/12 20:19,False,https://pbs.twimg.com/profile_banners/46144118...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1226134911...,@_____zac_____,100.0,27.652434,-1.457167,M,NaN,23.0
1,2,___aleia,7.650000e+17,en,_æ___ dad ___æ_,"Ohio, USA",BLACK. LIVES. MATTER.,False,466,388,3313,23454,8/14/16 19:45,False,https://pbs.twimg.com/profile_banners/76491083...,NaN,http://pbs.twimg.com/profile_images/1271280679...,@___aleia,100.0,24.111464,0.985713,F,NaN,19.0
3,4,___schaeffer___,1.257110e+09,en,Brenden Schaeffer,The Lou,Culver-Stockton College '20 ¢ Ô_Ô_Ô KM 1548...,False,811,660,26670,72205,3/10/13 14:07,False,https://pbs.twimg.com/profile_banners/12571104...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1268044218...,@___schaeffer___,59.0,35.518352,-3.591586,M,NaN,22.0
8,9,__drewc,1.050000e+18,en,drew,"New York, USA",_öÂ_öé _öÂ_ö_ _ç´Ù È \r\r26 #NewYork,False,27,128,181,113,10/17/18 4:12,False,https://pbs.twimg.com/profile_banners/10524118...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1090809548...,@__drewc,134.0,24.910635,1.969121,F,NaN,26.0
9,10,__EmilyRice__,3.797155e+09,en,em,"Marble Falls, TX",#TXST22,False,158,190,1219,2596,10/5/15 23:30,False,https://pbs.twimg.com/profile_banners/37971551...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/8415201103...,@__EmilyRice__,100.0,25.191925,2.382856,F,NaN,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,3272,ZoeBerrier,9.020000e+17,en,Zoâ _êâ,"Millersville, PA",MU 2021 (she/her)\r\rQueen of putting lipstick...,False,94,298,858,4428,8/28/17 21:03,False,https://pbs.twimg.com/profile_banners/90227535...,NaN,http://pbs.twimg.com/profile_images/1241199033...,@ZoeBerrier,100.0,22.585143,1.243141,F,F,20.0
3272,3273,ZoeCalamaco,3.214954e+09,no,Zoe _,San Angelo tx/ aspermont tx,Angelo state,False,475,318,6494,7475,4/28/15 0:24,False,https://pbs.twimg.com/profile_banners/32149543...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1277399838...,@ZoeCalamaco,100.0,21.348766,1.939069,F,F,21.0
3274,3275,ZoPeachy,9.890000e+17,en,Zobella Thee Alpha __ê_____´Ù__ ...,"New England/Boston, MA",Harlot for hire. FinDom. 27. Nonbinary. they/t...,False,396,635,3553,2075,4/25/18 3:51,False,https://pbs.twimg.com/profile_banners/98898876...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1153648270...,@ZoPeachy,100.0,34.894953,2.058720,F,NaN,27.0
3277,3278,zupercar1,4.220999e+09,en,Chris Avery _ê___êÈ,"Ann Arbor, MI",Comic Book Artist/ Film Major / I like space.....,False,121,1974,31208,70426,11/18/15 20:57,False,https://pbs.twimg.com/profile_banners/42209994...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1145126143...,@zupercar1,100.0,23.884346,-2.540524,M,NaN,21.0


In [ ]:
# Get authentication for tweepy from auth.txt
# auth.txt was uploaded before in order to use m3inference
keys = ''
with open(f'auth.txt', 'r') as reader:
  keys=reader.read()
keys = keys.split('\n')
for i in range(len(keys)):
  keys[i] = keys[i].split(':')[1];


In [ ]:
# Setup tweepy
import tweepy
api_key = keys[0]
api_secret = keys[1]
access_token = keys[2]
access_secret = keys[3]
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Extract latest 100 tweets using api.user_timeline for each user in Twitter_users_labeled_with_age_and_gender.csv

In [ ]:

import os
import time
from tqdm import tqdm


user = age_no_na["screen_name"].values.tolist()
age = age_no_na["human.labeled.age"].values.tolist()
age = [1 if a>=21 else 0 for a in age]

user_filter = []
age_filter = []
all_tweets = []

for i, userID in tqdm(enumerate(user)):
  try:
    tweets = api.user_timeline(screen_name=userID, count=100,include_rts = False, tweet_mode = 'extended')
    tweet_text = [tweet._json['full_text'] for tweet in tweets]
    all_tweets.append(tweet_text)
    user_filter.append(user[i])
    age_filter.append(age[i])
      
  except tweepy.TweepError as e:
    # print(e.response.status_code, e.reason)
    # the user not found error
    # print("screen_name that failed=",  userID)
    pass

print("Extraction Done")

# save the tweets for each user
print("the number of valid users", len(user_filter))
! mkdir tweets
for i, userID in tqdm(enumerate(user_filter)):
  filename = "tweets/" + str(i) + '.txt'
  
  with open(filename, 'w', encoding='utf-8') as f:
    f.write(userID + '\n')
    f.write(str(age_filter[i]) + '\n')
    for tw in all_tweets[i]:
      f.write(tw.replace('\n', ' ') + '\n')

In [ ]:
from glob import glob
from tqdm import tqdm
import numpy as np

user = []
age = []
tweets = []
count_len = []
count_num = []

saved_pths = glob('tweets/*.txt')
for filename in tqdm(saved_pths):
  with open(filename, 'r') as f:
    lines = f.readlines()
    tweet = [line.strip() for line in lines[2:]]
    if len(tweet) == 0:
      continue
    user.append(lines[0].strip())
    age.append(int(lines[1].strip()))
    count_num.append(len(tweet))
    count_len.extend([len(t.split()) for t in tweet])
    tweets.append(tweet)
# print(np.mean(count_len), max(count_len), min(count_len))
# print(np.mean(count_num), max(count_num), min(count_num))

100%|██████████| 1186/1186 [09:24<00:00,  2.10it/s]


the accuary of the Lexicon.age.prediction

In [ ]:
# age_no_na["screen_name"].values
a1 = age_no_na["human.labeled.age"].values >= 21
a2 = age_no_na["Lexicon.age.prediction"].values >= 21
b1 = a1 == a2
print("the accuary of the Lexicon.age.prediction")
sum(b1)/len(b1)

the accuary of the Lexicon.age.prediction


0.6387912860154603

In [ ]:
! pip  install transformers


Building the model

In [ ]:
from transformers import BertModel
import torch.nn as nn
from torch.nn import BCELoss

class Model(nn.Module):
    def __init__(self, config, model_name_or_path, num_labels=1):
        super().__init__()

        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained(model_name_or_path)
        self.dropout = nn.Dropout(0)
        self.classifier = nn.Sequential(nn.Linear(config.hidden_size*4, config.hidden_size // 2), nn.GELU(), nn.Linear(config.hidden_size // 2, self.num_labels), nn.Sigmoid())
        # self.freeze_params()

    # def freeze_params(self):
    #   for p in self.bert.parameters():
    #     p.requires_grad=False

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        output_hidden_states=True,
        return_dict=True,
        labels=None,
    ):

        output = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        x = torch.cat(output["hidden_states"][-4:], dim=-1)[:,0,:].mean(dim=0).unsqueeze(0) 
        # x = output["last_hidden_state"].mean(dim=1).mean(dim=0).unsqueeze(0)
        # x = output["pooler_output"].mean(dim=0).unsqueeze(0)    

        x = self.dropout(x)

        logits = self.classifier(x)

        outputs = (logits,)

        if labels is not None:
            loss_fct = BCELoss()
            loss = loss_fct(logits.view(-1), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs 

Training the model

In [ ]:
import random
import torch
from tqdm import tqdm
from transformers import BertTokenizer, BertConfig


### hyper parameters
learn_rate = 5e-6
epoch_num = 10
max_seq_len = 18
print_step = 100
seed = 2021
accumulation_steps = 10

model_name_or_path = "bert-base-uncased"
finetune_model_path = 'finetune_model.pth'

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# split train and test dataset
num_sample = len(user)
test_inx = random.sample(list(range(num_sample)),num_sample//10)
train_tw = []
test_tw = []
train_age = []
test_age = []
for i in range(num_sample):
  if i in test_inx:
    test_tw.append(tweets[i])
    test_age.append(age[i])
  else:
    train_tw.append(tweets[i])
    train_age.append(age[i])
print("the number of train:", len(train_age))
print("the number of test:", len(test_age))


# load model,  optimizer
config = BertConfig.from_pretrained(model_name_or_path)
tokenizer = BertTokenizer.from_pretrained(model_name_or_path)
model = Model(config, model_name_or_path)
model.to(device)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learn_rate)

# print the paramsters are expected to be updated
# for n, p in model.named_parameters():
#   if p.requires_grad:
#     print(n)

# start training 
best_score = 0
current_step = 0
print_loss = []

print()
for epoch in range(epoch_num):
  print("========= Epoch", epoch+1)
  # train
  epoch_loss = []
  model.train()
  for i, tw in enumerate(train_tw):
    encoded_input = tokenizer(tw, return_tensors='pt', max_length=max_seq_len, padding=True, truncation=True)
    input_ids = encoded_input['input_ids'].to(device)
    token_type_ids = encoded_input['token_type_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)
    label = torch.Tensor([train_age[i]]).to(device)

    loss = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=label)[0] 
    print_loss.append(loss.item())
    epoch_loss.append(loss.item())
    current_step += 1

    loss /= accumulation_steps
    loss.backward()

    if current_step % accumulation_steps == 0:
      optimizer.step()
      optimizer.zero_grad()
    
    if current_step % print_step == 0:
      print("mean train loss of 100 steps in the %d step: %.4f" % (current_step, sum(print_loss)/print_step))
      print_loss.clear()
  print("mean train loss in the %d epoch: %.4f" % (epoch+1, sum(epoch_loss)/len(epoch_loss)))
  print()

  # test
  model.eval()
  predict = np.array([])
  for i, tw in enumerate(test_tw):
    encoded_input = tokenizer(tw, return_tensors='pt', max_length=max_seq_len, padding=True, truncation=True)
    input_ids = encoded_input['input_ids'].to(device)
    token_type_ids = encoded_input['token_type_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    output = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
    output = torch.squeeze(output, -1).cpu().data.numpy() >= 0.5
    predict = np.append(predict, output.astype(int))
  ground = np.array(test_age)
  compr = predict == ground
  acc = np.sum(compr)/len(test_tw)
  if acc > best_score:
    best_score = acc
    torch.save(model.state_dict(), finetune_model_path)
  print("epoch %d   test acc : %.6f   best acc : %.6f" % (epoch+1, acc, best_score))
  print()
  print()


the number of train: 1064
the number of test: 118

========= Epoch 1
mean train loss of 100 steps in the 100 step: 0.6734
mean train loss of 100 steps in the 200 step: 0.6819
mean train loss of 100 steps in the 300 step: 0.6494
mean train loss of 100 steps in the 400 step: 0.6549
mean train loss of 100 steps in the 500 step: 0.6844
mean train loss of 100 steps in the 600 step: 0.6583
mean train loss of 100 steps in the 700 step: 0.6457
mean train loss of 100 steps in the 800 step: 0.6541
mean train loss of 100 steps in the 900 step: 0.6272
mean train loss of 100 steps in the 1000 step: 0.6840
mean train loss in the 1 epoch: 0.6618

epoch 1   test acc : 0.661017   best acc : 0.661017


========= Epoch 2
mean train loss of 100 steps in the 1100 step: 0.6580
mean train loss of 100 steps in the 1200 step: 0.6750
mean train loss of 100 steps in the 1300 step: 0.6724
mean train loss of 100 steps in the 1400 step: 0.6413
mean train loss of 100 steps in the 1500 step: 0.6656
mean train loss of

Predicting for the users in Twitter_users_labeled_with_age_and_gender.csv

In [ ]:
import random
import torch
from tqdm import tqdm
from transformers import BertTokenizer, BertConfig


# hyper parameters
max_seq_len = 18
model_name_or_path = "bert-base-uncased"
finetune_model_path = 'finetune_model.pth'

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")


# load model,  optimizer
config = BertConfig.from_pretrained(model_name_or_path)
tokenizer = BertTokenizer.from_pretrained(model_name_or_path)
model = Model(config, model_name_or_path)
model.load_state_dict(torch.load(finetune_model_path))
model.to(device)

# start predicting
model.eval()
predict = np.array([])
for i, tw in tqdm(enumerate(tweets), mininterval=2, desc=' - Predicting ' + str(len(age)) + 'it', leave=False):
  encoded_input = tokenizer(tw, return_tensors='pt', max_length=max_seq_len, padding=True, truncation=True)
  input_ids = encoded_input['input_ids'].to(device)
  token_type_ids = encoded_input['token_type_ids'].to(device)
  attention_mask = encoded_input['attention_mask'].to(device)

  output = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
  output = torch.squeeze(output, -1).cpu().data.numpy() >= 0.5
  predict = np.append(predict, output.astype(int))
ground = np.array(age)
compr = predict == ground
acc = np.sum(compr)/len(tweets)
print("total acc : %.4f" % acc)

Writing predictions into 'Twitter_users_labeled_prediction.csv'



In [ ]:
bert_age_prediction=[]
num_tweets_used=[]

for name in df_labeled["screen_name"]:
  if name not in user:
    bert_age_prediction.append(None)
    num_tweets_used.append(None)
  else:
    inx = user.index(name)
    if predict[inx]==1:
      ans = ">=21"
    else:
      ans = "<21"
    bert_age_prediction.append(ans)
    num_tweets_used.append(len(tweets[inx]))

df_labeled["bert.age.prediction"] = bert_age_prediction
df_labeled["num.tweets.used.bert.prediction"] = num_tweets_used
 
df_labeled.to_csv('Twitter_users_labeled_prediction.csv', index=True, header=True)


In [ ]:
df_labeled

,Unnamed: 0,screen_name,user_id,lang,name,location,description,protected,followers_count,friends_count,statuses_count,favourites_count,account_created_at,verified,profile_banner_url,profile_background_url,profile_image_url,user.name,num.tweets.used.Lexicon.prediction,Lexicon.age.prediction,Lexicon.gender.prediction..index.,lexicon.gender.prediction,human.labeled.gender,human.labeled.age,bert.age.prediction,num.tweets.used.bert.prediction
0,1,_____zac_____,4.614412e+08,en,zac ¢,"Maryland, USA",_____Û___È_Ü´Ù,False,208,178,27912,4179,1/11/12 20:19,False,https://pbs.twimg.com/profile_banners/46144118...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1226134911...,@_____zac_____,100.0,27.652434,-1.457167,M,NaN,23.0,>=21,42.0
1,2,___aleia,7.650000e+17,en,_æ___ dad ___æ_,"Ohio, USA",BLACK. LIVES. MATTER.,False,466,388,3313,23454,8/14/16 19:45,False,https://pbs.twimg.com/profile_banners/76491083...,NaN,http://pbs.twimg.com/profile_images/1271280679...,@___aleia,100.0,24.111464,0.985713,F,NaN,19.0,<21,62.0
2,3,___Dals,4.361882e+08,it,_ê_ê___ê__ê_ __,NJ,NaN,False,3744,1465,97563,22499,12/13/11 22:07,False,https://pbs.twimg.com/profile_banners/43618822...,http://abs.twimg.com/images/themes/theme14/bg.gif,http://pbs.twimg.com/profile_images/1267852466...,@___Dals,100.0,30.628957,0.734656,F,F,NaN,None,NaN
3,4,___schaeffer___,1.257110e+09,en,Brenden Schaeffer,The Lou,Culver-Stockton College '20 ¢ Ô_Ô_Ô KM 1548...,False,811,660,26670,72205,3/10/13 14:07,False,https://pbs.twimg.com/profile_banners/12571104...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1268044218...,@___schaeffer___,59.0,35.518352,-3.591586,M,NaN,22.0,>=21,5.0
4,5,__andresiscool,1.030000e+18,en,Andres Navarro,NaN,NaN,False,245,184,8731,18574,8/27/18 5:21,False,https://pbs.twimg.com/profile_banners/10339476...,NaN,http://pbs.twimg.com/profile_images/1272229718...,@__andresiscool,100.0,21.014222,-1.623685,M,M,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,3276,ZTheBest33,1.040000e+18,en,Adam. BLM.,northeast elitelism,"AOC/GND/Bern supporter| IFB | very biased | ""W...",False,345,751,17495,33147,9/7/18 11:09,False,https://pbs.twimg.com/profile_banners/10380214...,NaN,http://pbs.twimg.com/profile_images/1232111725...,@ZTheBest33,100.0,37.946711,-2.126250,M,M,NaN,None,NaN
3276,3277,ztran53,3.307187e+09,en,Zachary Tranter,"New York, NY",he/him/his,False,44,148,825,3129,6/3/15 16:58,False,https://pbs.twimg.com/profile_banners/33071871...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/6061595328...,@ztran53,100.0,34.509457,-2.469680,M,M,NaN,None,NaN
3277,3278,zupercar1,4.220999e+09,en,Chris Avery _ê___êÈ,"Ann Arbor, MI",Comic Book Artist/ Film Major / I like space.....,False,121,1974,31208,70426,11/18/15 20:57,False,https://pbs.twimg.com/profile_banners/42209994...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1145126143...,@zupercar1,100.0,23.884346,-2.540524,M,NaN,21.0,None,NaN
3278,3279,ZWHITE93,4.193607e+08,en,ZWH!tE,TX_´ÙPA,My life is centered around competition and cur...,False,593,528,69595,20793,11/23/11 8:06,False,https://pbs.twimg.com/profile_banners/41936067...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1229137769...,@ZWHITE93,100.0,32.228124,-0.860006,M,M,NaN,None,NaN


Extract latest 100 tweets using api.user_timeline for each user in Twitter_user_handles_to_predict.csv

In [ ]:
import os
import time
from tqdm import tqdm

pre_user = df_no_na["Username"].values.tolist()

pre_user_filter = []
pre_all_tweets = []

for i, userID in tqdm(enumerate(pre_user)):
  try:
    tweets = api.user_timeline(screen_name=userID, count=100,include_rts = False, tweet_mode = 'extended')
    tweet_text = [tweet._json['full_text'] for tweet in tweets]
    pre_all_tweets.append(tweet_text)
    pre_user_filter.append(pre_user[i])
      
  except tweepy.TweepError as e:
    # print(e.response.status_code, e.reason)
    # the user not found error
    # print("screen_name that failed=",  userID)
    pass

print("Extraction Done")

# save the tweets for each user
print("the number of valid users to predicted", len(pre_user_filter))
! mkdir pretweets
for i, userID in tqdm(enumerate(pre_user_filter)):
  filename = "pretweets/" + str(i) + '.txt'
  
  with open(filename, 'w', encoding='utf-8') as f:
    f.write(userID + '\n')
    for tw in pre_all_tweets[i]:
      f.write(tw.replace('\n', ' ') + '\n')

In [ ]:
from glob import glob
from tqdm import tqdm
import numpy as np
import json

tw_json = {}
pre_user = []
pre_tweets = []

saved_pths = glob('pretweets/*.txt')
# print(len(saved_pths))

for filename in tqdm(saved_pths):
  with open(filename, 'r') as f:
    lines = f.readlines()
    tweet = [line.strip() for line in lines[1:]]
    if len(tweet) == 0:
      continue
    pre_user.append(lines[0].strip())
    pre_tweets.append(tweet)
    tw_json[lines[0].strip()] = tweet

with open("pretweets.json", "w", encoding ="utf8") as f:
  json.dump(tw_json, f, ensure_ascii=False)


Predicting for the users in Twitter_user_handles_to_predict.csv


In [ ]:
import numpy as np
import json
import random
import torch
from tqdm import tqdm
from transformers import BertTokenizer, BertConfig

# load data
tw_json = json.load(open("pretweets.json", "r", encoding ="utf8"))
pre_user = list(tw_json.keys())

# hyper parameters
max_seq_len = 18
model_name_or_path = "bert-base-uncased"
finetune_model_path = 'finetune_model.pth'

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")


# load model
config = BertConfig.from_pretrained(model_name_or_path)
tokenizer = BertTokenizer.from_pretrained(model_name_or_path)
model = Model(config, model_name_or_path)
model.load_state_dict(torch.load(finetune_model_path))
model.to(device)

# start predicting
model.eval()
predict = np.array([])
for i, u in tqdm(enumerate(pre_user), mininterval=2, desc=' - Predicting ' + str(len(pre_user)) + 'it', leave=False):
  tw = tw_json[u]
  encoded_input = tokenizer(tw, return_tensors='pt', max_length=max_seq_len, padding=True, truncation=True)
  input_ids = encoded_input['input_ids'].to(device)
  token_type_ids = encoded_input['token_type_ids'].to(device)
  attention_mask = encoded_input['attention_mask'].to(device)

  output = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
  output = torch.squeeze(output, -1).cpu().data.numpy() >= 0.5
  predict = np.append(predict, output.astype(int))

Writing predictions into Twitter_user_handles_to_prediction.csv

In [ ]:
bert_age_prediction=[]
num_tweets_used=[]

for name in df["Username"]:
  if name not in pre_user:
    bert_age_prediction.append(None)
    num_tweets_used.append(None)
  else:
    inx = pre_user.index(name)
    if predict[inx]==1:
      ans = ">=21"
    else:
      ans = "<21"
    bert_age_prediction.append(ans)
    num_tweets_used.append(len(tw_json[name]))

df["bert.age.prediction"] = bert_age_prediction
df["num.tweets.used.bert.prediction"] = num_tweets_used
 
df.to_csv('Twitter_user_handles_to_prediction.csv', index=True, header=True)


In [ ]:
df

,ID,Username,bert.age.prediction,num.tweets.used.bert.prediction
0,1,JayHolz410,>=21,90.0
1,2,kelechief,>=21,67.0
2,3,VicSpencer,>=21,72.0
3,4,ItsAlexDodson,>=21,38.0
4,5,xopinkvodka6,<21,43.0
...,...,...,...,...
25124,25125,13luvr,None,NaN
25125,25126,Racetac,>=21,42.0
25126,25127,MrNoFilter_100,>=21,81.0
25127,25128,OgheneNerojr,>=21,72.0
